In [1]:
# === MeCab 엔진 및 파이썬 바인딩 완벽 설치 (권장) ===
# Colab 런타임이 재시작될 때마다 이 셀을 가장 먼저 실행해주세요.

import sys # sys 모듈을 미리 임포트합니다.

print("단계 1/5: konlpy 설치 및 환경 업데이트...")
print("----------------------------------------------------------------")
!pip install konlpy  # konlpy 설치 (JPype1 포함)
!apt-get update      # 패키지 목록 업데이트
!apt-get install curl -y # curl 설치
print("----------------------------------------------------------------")

print("\n단계 2/5: MeCab 엔진 및 한국어 사전 설치 (konlpy 공식 스크립트 사용)...")
print("----------------------------------------------------------------")
# konlpy 공식 github에서 제공하는 MeCab 엔진 및 사전 설치 스크립트 실행
# 이 스크립트가 MeCab 본체 (C++ 라이브러리)와 mecab-ko-dic 사전을 /usr/local/ 경로에 설치합니다.
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
print("----------------------------------------------------------------")

print("\n단계 3/5: 기존 python-mecab-ko 제거 (깨끗한 재설치를 위해)...")
print("----------------------------------------------------------------")
# 이전 "Requirement already satisfied: python-mecab-ko" 메시지가 나왔지만,
# import가 안 되는 문제를 해결하기 위해 강제로 제거 후 재설치합니다.
!pip uninstall -y python-mecab-ko
print("----------------------------------------------------------------")

print("\n단계 4/5: MeCab 파이썬 바인딩 재설치 (가장 안정적인 'mecab-python3' 사용)...")
print("----------------------------------------------------------------")
# 이전 "subprocess-exited-with-error" 오류를 우회하고,
# 'import MeCab' 문제를 해결하기 위해 가장 일반적인 바인딩을 명시적으로 설치합니다.
!pip install mecab-python3
print("----------------------------------------------------------------")

print("\n단계 5/5: MeCab 설치 최종 진단 및 Tagger 초기화 테스트...")
print("----------------------------------------------------------------")
# MeCab 실행 파일 확인
!which mecab
# MeCab 버전 및 사전 경로 확인
!mecab -D # MeCab 엔진 및 사전 경로 확인

# 파이썬 내에서 MeCab 모듈 임포트 및 Tagger 초기화 테스트
print("\n파이썬 내 MeCab 모듈 임포트 및 Tagger 테스트:")
try:
    import MeCab # 이제 이 임포트가 성공해야 합니다.
    print('MeCab 파이썬 모듈 임포트 성공!')
    MECAB_DIC_PATH = '/usr/local/lib/mecab/dic/mecab-ko-dic/' # MeCab 사전 경로
    # MeCab.Tagger 초기화 시 mecabrc 경로 지정(-r 옵션)을 제거하여 MeCab이 자동으로 찾도록 합니다.
    tagger = MeCab.Tagger(f'-d {MECAB_DIC_PATH}')
    print('MeCab Tagger 객체 성공적으로 생성됨.')
except ImportError as e:
    print(f'**치명적 오류**: MeCab 파이썬 모듈 임포트 실패! - {e}')
    print('MeCab 설치에 문제가 있습니다. Colab 런타임을 재시작하고 다시 시도하거나, 다른 Colab 환경을 사용해보세요.')
    sys.exit(1) # 오류 발생 시 스크립트 종료
except RuntimeError as e:
    print(f'**치명적 오류**: MeCab Tagger 객체 생성 실패! - {e}')
    print('MeCab Tagger 초기화에 문제가 있습니다. 사전 경로를 확인하거나 MeCab 설치를 재확인하세요.')
    sys.exit(1) # 오류 발생 시 스크립트 종료
except Exception as e:
    print(f'**치명적 오류**: 예상치 못한 오류 발생: {e}')
    sys.exit(1) # 오류 발생 시 스크립트 종료
print("----------------------------------------------------------------")
print("\n모든 설치 및 진단 단계 완료. 위의 파이썬 테스트 결과에 따라 다음 진행 여부가 결정됩니다.")

단계 1/5: konlpy 설치 및 환경 업데이트...
----------------------------------------------------------------
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sour

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import MeCab
import sys
import json
from typing import List, Tuple, Dict, Any
import numpy as np
import time # 시간 측정을 위한 time 모듈 임포트

# --- 1. 환경 설정 및 유틸리티 함수 ---

# initialize_mecab_tagger 함수 수정: mecabrc_path 인자 사용 방식 변경 (이전 수정과 동일)
def initialize_mecab_tagger(dic_path: str) -> MeCab.Tagger:
    """
    MeCab Tagger를 초기화하고 반환합니다.
    Args:
        dic_path (str): MeCab 한국어 사전의 루트 디렉토리 경로.
    Returns:
        MeCab.Tagger: 초기화된 MeCab Tagger 객체.
    Raises:
        RuntimeError: Tagger 초기화에 실패할 경우 발생.
    """
    try:
        tagger = MeCab.Tagger(f'-d {dic_path}')
        print(f"MeCab Tagger가 성공적으로 초기화되었습니다.")
        print(f"  - 사전 경로: {dic_path}")
        return tagger
    except RuntimeError as e:
        print(f"MeCab Tagger 초기화 오류: {e}")
        print(f"  - 시도된 사전 경로: {dic_path}")
        print("MeCab Tagger 초기화에 실패했습니다. MeCab 설치 및 경로 설정을 다시 확인해주세요.")
        sys.exit(1) # 오류 발생 시 프로그램 종료


def tokenize_sentences(tagger: MeCab.Tagger, sentences: List[str]) -> List[List[str]]:
    """
    문장 리스트를 형태소 분석하여 토큰화된 문장 리스트를 반환합니다.
    """
    tokenized_sentences = []
    start_time = time.time()
    for i, sentence in enumerate(sentences):
        if i > 0 and i % 5000 == 0: # 5천개 문장마다 진행 상황 출력
            print(f"  - {i}/{len(sentences)} 문장 토큰화 완료 ({time.time() - start_time:.2f} 초)")
        if not sentence or not isinstance(sentence, str):
            continue
        try:
            parsed_lines = tagger.parse(sentence).strip().split('\n')
        except Exception as e:
            # print(f"  경고: 문장 토큰화 중 오류 발생: {sentence[:50]}... - {e}") # 디버깅용
            continue

        sentence_tokens = []
        for line in parsed_lines:
            if '\t' in line:
                word = line.split('\t')[0]
                if '+' in word and not word.startswith('+'):
                    word = word.split('+')[0]
                sentence_tokens.append(word)
        if sentence_tokens:
            tokenized_sentences.append(sentence_tokens)
    print(f"  - 모든 문장 토큰화 완료. 총 {len(tokenized_sentences)}개 유효 문장 ({time.time() - start_time:.2f} 초 소요)")
    return tokenized_sentences

def build_vocab(tokenized_sentences: List[List[str]], min_count: int = 5) -> Tuple[Dict[str, int], Dict[int, str], int]:
    """
    토큰화된 문장들로부터 단어 사전(vocab)을 구축합니다.
    """
    start_time = time.time()
    all_words = [word for sentence in tokenized_sentences for word in sentence]
    word_counts = Counter(all_words)

    filtered_word_list = [word for word, count in word_counts.items() if count >= min_count]
    word_list = sorted(filtered_word_list)

    word_to_idx = {word: i for i, word in enumerate(word_list)}
    idx_to_word = {i: word for word, i in word_to_idx.items()}
    vocab_size = len(word_to_idx)

    print(f"단어 사전 크기: {vocab_size} (최소 빈도 {min_count} 이상) ({time.time() - start_time:.2f} 초 소요)")
    return word_to_idx, idx_to_word, vocab_size

def create_cbow_dataset(
    tokenized_sentences: List[List[str]],
    word_to_idx: Dict[str, int],
    window_size: int = 1
) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    CBOW 모델 학습을 위한 컨텍스트-타겟 쌍 데이터셋을 생성합니다.
    """
    context_indices: List[List[int]] = []
    target_indices: List[int] = []

    start_time = time.time()
    for i, tokens in enumerate(tokenized_sentences):
        if i > 0 and i % 5000 == 0: # 5천개 문장마다 진행 상황 출력
            print(f"  - {i}/{len(tokenized_sentences)} 문장 CBOW 데이터 생성 진행 중 ({time.time() - start_time:.2f} 초)")

        if len(tokens) <= 2 * window_size:
            continue

        for j in range(window_size, len(tokens) - window_size):
            target_word = tokens[j]

            context_words = []
            for k in range(1, window_size + 1):
                context_words.append(tokens[j - k])
                context_words.append(tokens[j + k])

            if target_word in word_to_idx and all(word in word_to_idx for word in context_words):
                context_indices.append([word_to_idx[word] for word in context_words])
                target_indices.append(word_to_idx[target_word])

    if not context_indices:
        print("경고: CBOW 학습 데이터가 생성되지 않았습니다. 문장 길이, 윈도우 크기, 또는 단어 사전을 확인하세요.")
        return torch.tensor([]), torch.tensor([])

    contexts_tensor = torch.tensor(context_indices, dtype=torch.long)
    targets_tensor = torch.tensor(target_indices, dtype=torch.long)

    print(f"CBOW 컨텍스트 데이터 형태: {contexts_tensor.shape}")
    print(f"CBOW 타겟 데이터 형태: {targets_tensor.shape}")
    print(f"  - CBOW 학습 데이터 생성 완료 ({time.time() - start_time:.2f} 초 소요)")
    return contexts_tensor, targets_tensor

# --- 2. CBOW 모델 정의 ---

class CBOW(nn.Module):
    """
    Continuous Bag-of-Words (CBOW) 모델 구현.
    """
    def __init__(self, vocab_size: int, embedding_dim: int):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        embeds = self.embeddings(inputs)
        mean_embeds = embeds.mean(dim=1)
        output = self.linear(mean_embeds)
        return output

# --- 3. 학습 함수 정의 ---

def train_cbow_model(
    model: CBOW,
    contexts: torch.Tensor,
    targets: torch.Tensor,
    epochs: int = 100,
    learning_rate: float = 0.01,
    device: torch.device = torch.device('cpu')
) -> None:
    if contexts.numel() == 0:
        print("학습 데이터가 없어 모델 학습을 건너킵니다.")
        return

    model.to(device)
    contexts = contexts.to(device)
    targets = targets.to(device)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    print(f"\nCBOW 모델 학습 시작 (장치: {device})...")
    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        output = model(contexts)
        loss = loss_fn(output, targets)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f} ({time.time() - start_time:.2f} 초 경과)")
    print("CBOW 모델 학습 완료.")

def get_word_embedding(model: CBOW, word: str, word_to_idx: Dict[str, int]) -> np.ndarray:
    """
    특정 단어의 학습된 임베딩 벡터를 반환합니다.
    """
    if word not in word_to_idx:
        raise ValueError(f"'{word}'는 단어 사전에 없습니다.")

    word_id = word_to_idx[word]
    embedding = model.embeddings(torch.tensor([word_id], dtype=torch.long, device=model.embeddings.weight.device))
    return embedding.detach().cpu().numpy()

# --- 4. JSON 파일 로드 함수 ---

def load_sentences_from_json(file_path: str, max_sentences: int = -1) -> List[str]:
    """
    'train_original.json' 파일의 특정 구조에 맞춰 문장 데이터를 로드합니다.
    """
    sentences_list: List[str] = []
    start_time = time.time()
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

            if isinstance(data, dict):
                if "documents" in data and isinstance(data["documents"], list):
                    for doc_item in data["documents"]:
                        if max_sentences != -1 and len(sentences_list) >= max_sentences:
                            break

                        if isinstance(doc_item, dict) and "text" in doc_item:
                            text_content = doc_item["text"]
                            if isinstance(text_content, list):
                                for sentence_block in text_content:
                                    if isinstance(sentence_block, list):
                                        for sentence_obj in sentence_block:
                                            if max_sentences != -1 and len(sentences_list) >= max_sentences:
                                                break

                                            if isinstance(sentence_obj, dict) and "sentence" in sentence_obj:
                                                if isinstance(sentence_obj["sentence"], str):
                                                    sentences_list.append(sentence_obj["sentence"])
                                        if max_sentences != -1 and len(sentences_list) >= max_sentences:
                                            break
                else:
                    print("오류: JSON 파일의 최상위 딕셔너리에서 'documents' 키를 찾을 수 없거나 해당 키의 값이 리스트가 아닙니다.")
                    sys.exit(1)
            else:
                print(f"오류: JSON 파일의 최상위 구조가 딕셔너리가 아닙니다. 지원되지 않는 형식입니다: {type(data)}")
                sys.exit(1)

    except FileNotFoundError:
        print(f"오류: 파일을 찾을 수 없습니다 - {file_path}")
        sys.exit(1)
    except json.JSONDecodeError as e:
        print(f"오류: JSON 디코딩 실패 - {file_path}. 파일 내용이 유효한 JSON 형식이 아닐 수 있습니다: {e}")
        sys.exit(1)
    except Exception as e:
        print(f"파일 로드 중 알 수 없는 오류 발생: {e}")
        sys.exit(1)

    print(f"총 {len(sentences_list)}개의 문장을 '{file_path}'에서 로드했습니다. ({time.time() - start_time:.2f} 초 소요)")
    return sentences_list

# --- 5. 메인 실행 블록 ---

if __name__ == "__main__":
    # --- MeCab 설정 (코랩 환경에 맞춰 미리 설정됨) ---
    MECAB_DIC_PATH = "/usr/local/lib/mecab/dic/mecab-ko-dic/" # 코랩 MeCab 사전 경로

    # --- JSON 파일 경로 및 데이터 로드 설정 (코랩 로컬 업로드용) ---
    JSON_FILE_PATH = "/content/train_original.json" # <-- 코랩 로컬 업로드 시 이 경로를 사용합니다!

    # 테스트를 위해 문장 수를 획기적으로 줄였습니다.
    # 이 값을 50000으로 다시 늘리려면 이 테스트가 성공한 후에 시도하세요.
    MAX_SENTENCES_TO_LOAD = 5000 # <-- 일단 1000개 문장만 로드하여 테스트.
    MIN_WORD_COUNT = 5 # <-- 단어 사전에 포함할 단어의 최소 빈도.

    # --- PyTorch 장치 설정 ---
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"학습 장치: {device}")

    # 1. MeCab Tagger 초기화
    print("\n[1단계: MeCab Tagger 초기화]")
    start_time_step = time.time()
    mecab_tagger = initialize_mecab_tagger(MECAB_DIC_PATH)
    print(f"1단계 완료. ({time.time() - start_time_step:.2f} 초 소요)")

    # 2. JSON 파일에서 문장 데이터 로드
    print("\n[2단계: JSON 파일에서 문장 데이터 로드]")
    start_time_step = time.time()
    raw_sentences = load_sentences_from_json(JSON_FILE_PATH, MAX_SENTENCES_TO_LOAD)
    print(f"2단계 완료. ({time.time() - start_time_step:.2f} 초 소요)")

    # 3. 형태소 분석 및 토큰화
    print("\n[3단계: 형태소 분석 및 토큰화]")
    start_time_step = time.time()
    tokenized_data = tokenize_sentences(mecab_tagger, raw_sentences)
    if not tokenized_data:
        print("경고: 형태소 분석 결과 생성된 유효한 토큰 데이터가 없습니다. 원본 데이터 또는 MeCab 설정을 확인하세요.")
        sys.exit(1)
    print(f"\n[토큰화 샘플 (총 {len(tokenized_data)}개 문장 중 처음 5개)]")
    for i, s in enumerate(tokenized_data[:5]):
        print(f"  {i+1}: {s}")
    print(f"3단계 완료. ({time.time() - start_time_step:.2f} 초 소요)")


    # 4. 단어 사전 구축
    print("\n[4단계: 단어 사전 구축]")
    start_time_step = time.time()
    word_to_idx, idx_to_word, vocab_size = build_vocab(tokenized_data, min_count=MIN_WORD_COUNT)
    print(f"4단계 완료. ({time.time() - start_time_step:.2f} 초 소요)")

    # 5. CBOW 학습 데이터 생성
    print("\n[5단계: CBOW 학습 데이터 생성]")
    start_time_step = time.time()
    WINDOW_SIZE = 1 # 컨텍스트 윈도우 크기
    contexts_tensor, targets_tensor = create_cbow_dataset(tokenized_data, word_to_idx, WINDOW_SIZE)
    print(f"5단계 완료. ({time.time() - start_time_step:.2f} 초 소요)")

    # 6. CBOW 모델 학습
    print("\n[6단계: CBOW 모델 학습]")
    EMBEDDING_DIM = 10 # 임베딩 차원
    if vocab_size > 0 and contexts_tensor.numel() > 0: # 단어 사전과 학습 데이터가 있을 때만 모델 생성 및 학습
        cbow_model = CBOW(vocab_size, EMBEDDING_DIM)
        train_cbow_model(cbow_model, contexts_tensor, targets_tensor,
                         epochs=300, learning_rate=0.005, device=device) # 에포크와 학습률 조정

        # 7. 특정 단어의 임베딩 확인 (예시 단어 - 법률 문서 관련)
        print("\n[7단계: 테스트 단어 임베딩 확인]")
        test_words = ["법률", "판결", "계약", "소송", "원고", "피고", "재판", "사건", "권리"]
        print("\n[테스트 단어 임베딩 확인]")
        for word in test_words:
            try:
                embedding_vector = get_word_embedding(cbow_model, word, word_to_idx)
                print(f"'{word}'의 임베딩: {embedding_vector}")
            except ValueError as e:
                print(f"{e}")
    else:
        print("\n단어 사전이 비어있거나 학습 데이터가 없어 모델 학습을 진행할 수 없습니다.")

    print("\n모든 코드 실행 완료!")

학습 장치: cpu

[1단계: MeCab Tagger 초기화]
MeCab Tagger가 성공적으로 초기화되었습니다.
  - 사전 경로: /usr/local/lib/mecab/dic/mecab-ko-dic/
1단계 완료. (0.02 초 소요)

[2단계: JSON 파일에서 문장 데이터 로드]
총 5000개의 문장을 '/content/train_original.json'에서 로드했습니다. (1.70 초 소요)
2단계 완료. (1.81 초 소요)

[3단계: 형태소 분석 및 토큰화]
  - 모든 문장 토큰화 완료. 총 5000개 유효 문장 (1.80 초 소요)

[토큰화 샘플 (총 5000개 문장 중 처음 5개)]
  1: ['원고', '가', '소속', '회사', '의', '노동조합', '에서', '분규', '가', '발생', '하', '자', '노조', '활동', '을', '구실', '로', '정상', '적', '인', '근무', '를', '해태', '하', '고', ',']
  2: ['노조', '조합장', '이', '사임', '한', '경우', ',']
  3: ['노동조합', '규약', '에', '동', '조합장', '의', '직무', '를', '대행', '할', '자', '를', '규정', '해', '두', '고', '있', '음', '에', '도', '원고', '자신', '이', '주동', '하', '여', '노조', '자치', '수습', '대책', '위원회', '를', '구성', '하', '여', '그', '위원장', '으로', '피선', '되', '어', '근무', '시간', '중', '에', '도', '노조', '활동', '을', '벌여', '운수', '업체', '인', '소속', '회사', '의', '업무', '에', '지장', '을', '초래', '하', '고']
  4: ['종업원', '들', '에게', '도', '나쁜', '영향', '을', '끼쳐', '소속', '회사', '가', '취업규칙', '을', '위반', '하', '고']
  5